## Importing the necessary libraries 

In [1]:
## Import Neural nets related libraries
from __future__ import print_function
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets , transforms 
import torchvision
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter

In [6]:
## Parms for Neural network training
batch_size=64

## Preparing data for building the model 

In [7]:
train_dataset = datasets.MNIST(root='./mnist_data/',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

In [8]:
test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform = transforms.ToTensor()
                              )

In [9]:
## Let's look at train & test data
print('=====Train dataset=====')
print(train_dataset)
print('\n\n=====Test dataset=====')
print(test_dataset)

=====Train dataset=====
Dataset MNIST
    Number of datapoints: 60000
    Root location: ./mnist_data/
    Split: Train
    StandardTransform
Transform: ToTensor()


=====Test dataset=====
Dataset MNIST
    Number of datapoints: 10000
    Root location: ./mnist_data/
    Split: Test
    StandardTransform
Transform: ToTensor()


## Preparing the train & test data pipeline

In [10]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [19]:
'''
# create grid of images
dataiter = iter(train_loader)
images, labels = dataiter.next()
# create grid of images
img_grid = torchvision.utils.make_grid(images)
# write to tensorboard
writer.add_image('mnist_images', img_grid)
'''

## Design - NeuralNet Architecture 

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520) ## 28*28 - Image - so input size is 784
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):   ## Connection all the layers with activation function in the forword loop
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)   # Return the logit layer - o/p will be 10 possibe values # criterion = nn.CrossEntropyLoss() will do softmax on 5th Layer output

In [16]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

## Define training loop

In [17]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(\
                epoch, batch_idx * len(data), len(train_loader.dataset),\
                100. * batch_idx / len(train_loader), loss))

In [20]:
def test():
    model.eval()
    test_loss = 0 
    correct = 0 
    for data,target in test_loader:
        data,target = Variable(data),Variable(target)
        output = model(data)
        test_loss += criterion(output,target)
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [25]:
import os
curr_path = os.getcwd()

In [24]:
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.018464
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.078657
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.031677
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.022520
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.055087
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.045715
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.060606
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.030407
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.028390
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.053317
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.022413
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.166055
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.057904
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.033541
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.029422
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.068275
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.015747
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.032319
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.017177
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.041102
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.077468
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.006139
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.044669
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.157836
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.040790
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.104513
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.062324
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.105963
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.028203
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.155903
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.020597
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.030799
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.021471
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.053243
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.049270
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.016391
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.036544
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.087356
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.006233
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.016077


Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.041192
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.133260
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.019370
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.022461
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.016524
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.019144
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.038635
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.062267
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.055320
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.014931
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.098557
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.010043
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.015334
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.004088
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.016224
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.032790
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.007286
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.069129
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.033877
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.029415


Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.027979
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.015968
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.089172
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.119848
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.072744
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.005544
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.043249
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.078444
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.008803
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.038100
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.021321
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.005983
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.020147
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.046475
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.014470
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.059438
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.008936
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.016153
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.030055
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.036441


Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.020582

Test set: Average loss: 0.0013, Accuracy: 9758/10000 (97%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.009250
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.003157
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.023673
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.026737
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.061647
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.023340
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.029035
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.006809
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.037121
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.063329
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.009625
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.005907
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.046126
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.014925
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.014998
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.035955
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.019236
Train Epoch: 8 [10880/60000 (18%)]	Loss: 0.008779
Train Epoch: 8 [11

Train Epoch: 9 [43520/60000 (72%)]	Loss: 0.006328
Train Epoch: 9 [44160/60000 (74%)]	Loss: 0.003269
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.018533
Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.049608
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.015038
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.036020
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.029309
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.003098
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.014238
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.027729
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.041142
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.011671
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.050426
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.009946
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.061167
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.021435
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.004094
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.007336
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.016251
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.119259


NameError: name 'curr_path' is not defined

In [27]:
## Saving the model
torch.save(model.state_dict(), curr_path+'/'+'mnist.pytorch_model')